In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import re
import csv
import random
import string
import multiprocessing
import itertools
import random
import csv
import statistics
import string
import random
from tensorflow.keras.regularizers import l2

In [ ]:
ROUND_KEY = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

INT_NUM = 2 * 4
MASTER_KEY_SIZE = 2


def key_add(val, rk, num):  # C에서의 RK+(i*8)를 num 인수에 받아서 처리
    # 호출될 때마다 num값에 따라 rk array index가 바뀜
    val[0] ^= rk[0+8*num]
    val[1] ^= rk[1+8*num]
    val[2] ^= rk[2+8*num]
    val[3] ^= rk[3+8*num]
    val[4] ^= rk[4+8*num]
    val[5] ^= rk[5+8*num]
    val[6] ^= rk[6+8*num]
    val[7] ^= rk[7+8*num]


def sbox(X):
    T = [0, 0, 0]
    # (MSB: x[7], LSB: x[0])
    # Input: x[7], x[6], x[5], x[4], x[3], x[2], x[1], x[0]
    # S5_1

    X[5] ^= (X[7] & X[6]) & 0xFF
    X[4] ^= (X[3] & X[5]) & 0xFF
    X[7] ^= X[4] & 0xFF
    X[6] ^= X[3] & 0xFF
    X[3] ^= (X[4] | X[5]) & 0xFF
    X[5] ^= X[7] & 0xFF
    X[4] ^= (X[5] & X[6]) & 0xFF

    # S3
    X[2] ^= X[1] & X[0] & 0xFF
    X[0] ^= X[2] | X[1] & 0xFF
    X[1] ^= X[2] | X[0] & 0xFF
    X[2] = ~X[2] & 0xFF

    # Extend XOR
    X[7] ^= X[1] & 0xFF
    X[3] ^= X[2] & 0xFF
    X[4] ^= X[0] & 0xFF

    # S5_2
    T[0] = X[7] & 0xFF
    T[1] = X[3] & 0xFF
    T[2] = X[4] & 0xFF
    X[6] ^= (T[0] & X[5]) & 0xFF
    T[0] ^= X[6] & 0xFF
    X[6] ^= (T[2] | T[1]) & 0xFF
    T[1] ^= X[5] & 0xFF
    X[5] ^= (X[6] | T[2]) & 0xFF
    T[2] ^= (T[1] & T[0]) & 0xFF

    # Truncate XOR and bit change
    X[2] ^= T[0] & 0xFF
    T[0] = X[1] ^ T[2] & 0xFF
    X[1] = X[0] ^ T[1] & 0xFF
    X[0] = X[7] & 0xFF
    X[7] = T[0] & 0xFF
    T[1] = X[3] & 0xFF
    X[3] = X[6] & 0xFF
    X[6] = T[1] & 0xFF
    T[2] = X[4] & 0xFF
    X[4] = X[5] & 0xFF
    X[5] = T[2] & 0xFF

    # Output: (MSb) x[7], x[6], x[5], x[4], x[3], x[2], x[1], x[0] (LSb)


def pbox(X):
    # & 0XFF를 추가시켜준 이유 : C에서는 8비트로 설정해놓으면 계산하면서 자동으로 8비트 넘으면 잘라버리는데
    # 파이썬은 고려 안해줘서 25a5처럼 16비트로 나옴. 우리는 뒤의 8비트인 a5만 필요함. 그래서 앞의 8비트를 잘랐음.
    X[1] = ((X[1] << 7) | (X[1] >> 1)) & 0XFF
    X[2] = ((X[2] << 4) | (X[2] >> 4)) & 0XFF
    X[3] = ((X[3] << 3) | (X[3] >> 5)) & 0XFF
    X[4] = ((X[4] << 6) | (X[4] >> 2)) & 0XFF
    X[5] = ((X[5] << 5) | (X[5] >> 3)) & 0XFF
    X[6] = ((X[6] << 1) | (X[6] >> 7)) & 0XFF
    X[7] = ((X[7] << 2) | (X[7] >> 6)) & 0XFF


def ENC(PLAIN_TEXT, ROUND_KEY, ROUND):
    P = []
    RK = [] 
    
    RK = ROUND_KEY
    
    for i in range(2):  # P 비트 쪼개기 작업
        for j in range(4):
            P.append(PLAIN_TEXT[i] & 0XFF)
            PLAIN_TEXT[i] = PLAIN_TEXT[i] >> 8

    key_add(P, RK, 0)

    for i in range(1, ROUND+1):
        sbox(P)
        pbox(P)
        key_add(P, RK, i)

    C_l = (P[7]<<24) + (P[6]<<16) + (P[5]<<8) + P[4]
    C_r = (P[3]<<24) + (P[2]<<16) + (P[1]<<8) + P[0]

    return C_l, C_r

def ROUND_KEY_GEN(MASTER_KEY):
    RCON = 0
    ROUND = 13

    for i in range(0, ROUND+1):
        for j in range(0, INT_NUM):
            ROUND_KEY[INT_NUM * i + j] = MASTER_KEY[(INT_NUM * i + j) % (MASTER_KEY_SIZE * INT_NUM)]
        ROUND_KEY[INT_NUM * i] ^= RCON
        RCON += 1

    return ROUND_KEY

In [ ]:
import numpy as np
from os import urandom


X = []

#diff=(0x88000880,0x08088000)
diff=(0x01000101,0x00010100)
# diff=(0x00000000,0x00008000)
# diff=(0x00000000,0x00080080)
# diff=(0x20111120,0x00800080)
# diff=(0x40410040,0x8101c080)
# diff=(0x00001010,0x00100000)
# diff=(0x00000000,0x80000000)
# diff=(0x00010000,0x04084000)

nr = 2
n = 3000000

def Append(k, text):
    for i in range(0, len(text)):
        X[k].append(list(text)[i])

def Append2(k, text):
        X[k].append(text)        

#baseline training data generator


l = 0

MASTER_KEY = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
plain0l = np.zeros(shape=(n,),dtype=int)
plain0r = np.zeros(shape=(n,),dtype=int)
plain1l = np.zeros(shape=(n,),dtype=int)
plain1r = np.zeros(shape=(n,),dtype=int)


for i in range(n):
  plain0l[i] = random.randint(0, (2**32)-1) # 8-bit
  plain0r[i] = random.randint(0, (2**32)-1) # 8-bit

  plain1l[i] = plain0l[i] ^ diff[0];
  plain1r[i] = plain0r[i] ^ diff[1];

Y = np.ones(shape=(n,)) # cipher

for i in range(int(n/2)):
  plain1l[i] = random.randint(0, (2**32)-1)
  plain1r[i] = random.randint(0, (2**32)-1)
  Y[i] = 0 # random 

plain0 = np.zeros(shape=(n,2),dtype=int) 
plain1 = np.zeros(shape=(n,2),dtype=int)
ctdata0 = np.zeros(shape=(n,),dtype=int)
ctdata1 = np.zeros(shape=(n,),dtype=int)

ctdata0r = np.zeros(shape=(n,),dtype=int)
ctdata0l = np.zeros(shape=(n,),dtype=int)
ctdata1r = np.zeros(shape=(n,),dtype=int)
ctdata1l = np.zeros(shape=(n,),dtype=int)

for i in range(n):

  plain0[i][0] = plain0r[i]
  plain0[i][1] = plain0l[i]

  plain1[i][0] = plain1r[i]
  plain1[i][1] = plain1l[i]

  #plain0[i] = int((2**32)*(plain0l[i]) + plain0r[i]) 
  #plain1[i] = int((2**32)*(plain1l[i]) + plain1r[i])

  #ks = random.randint(0, (2**128)-1)

  for j in range(0, 16):  # MASTER_KEY 생성
      MASTER_KEY[j] = random.randint(0, 255)

  ROUND_KEY = ROUND_KEY_GEN(MASTER_KEY)
  ctdata0l[i], ctdata0r[i] = ENC(plain0[i], ROUND_KEY, nr) # 8비트씩 8개 - > 이걸 정수로 하고싶음 
  ctdata1l[i], ctdata1r[i] = ENC(plain1[i], ROUND_KEY, nr) # 8비트씩 8개

  c0l = str(bin(ctdata0l[i])).split('0b')[1].zfill(32)
  c0r = str(bin(ctdata0r[i])).split('0b')[1].zfill(32)
  c1l = str(bin(ctdata1l[i])).split('0b')[1].zfill(32)
  c1r = str(bin(ctdata1r[i])).split('0b')[1].zfill(32)

  X.append([l])
  Append(l,c0l)
  Append(l,c0r)
  Append(l,c1l)
  Append(l,c1r)
  Append2(l, Y[i])

  '''
  X.append([l])
  Append(l,c0l)
  Append(l,c0r)
  Append(l,c1l)
  Append(l,c1r)
  Append2(l, Y[i])
  '''

  l = l+1


################## 파일 저장 코드 지위기 ##################
f = open("/content/drive/MyDrive/PIPO-dataset/DC/PIPO-DC-2round-300-diff1.csv", "wt", newline="")
csvwriter = csv.writer(f)
for i in range(0, n):
    csvwriter.writerow(X[i])

print("end")
f.close()


end


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/PIPO-dataset/DC/PIPO-DC-2round-300-diff1.csv',header=None).values

labels = []
datas = []
n_bit = 64
div = 2*n_bit +1



#formatting
for i in range(0,len(data)):
  labels.append(data[i][div:])
  datas.append(data[i][1:div])

datas = np.asarray(datas)
labels = np.asarray(labels)

#print(datas)

x_train, x_val, y_train, y_val = train_test_split(datas, labels, test_size=0.4)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.05)

print(len(x_train))
print(len(x_val))
print(len(x_test))


1710000
1200000
90000
[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [ ]:
inp = tf.keras.layers.Input(shape=(128, ))

x = tf.keras.layers.Dense(128, activation = 'relu',kernel_regularizer=l2(0.001))(inp)
shortcut = x
for i in range(5):
    x = tf.keras.layers.Dense(128, activation = 'relu',kernel_regularizer=l2(0.001))(shortcut)
    x = tf.keras.layers.Dense(128, activation = 'relu',kernel_regularizer=l2(0.001))(x)
    shortcut = tf.keras.layers.Add()([shortcut, x])

out = tf.keras.layers.Dense(1, activation = 'sigmoid')(shortcut)
model = tf.keras.Model(inputs=inp, outputs=out)

model.summary()

s = 100 * len(x_train) // 8

lr = keras.optimizers.schedules.ExponentialDecay(0.0001, s, 0.001)
opt = keras.optimizers.Adam(lr)

model.compile(optimizer=opt,
              loss='binary_crossentropy', #binary_crossentropy
              metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          16512       ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 128)          16512       ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 128)          16512       ['dense_1[0][0]']                
                                                                                              

In [ ]:
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=50, batch_size = 8) # 100, 32

Epoch 1/50
213750/213750 [==============================] - 970s 5ms/step - loss: 0.3300 - accuracy: 0.9187 - val_loss: 0.2037 - val_accuracy: 0.9438
Epoch 2/50
213750/213750 [==============================] - 959s 4ms/step - loss: 0.1818 - accuracy: 0.9496 - val_loss: 0.1620 - val_accuracy: 0.9550
Epoch 3/50
213750/213750 [==============================] - 965s 5ms/step - loss: 0.1373 - accuracy: 0.9639 - val_loss: 0.1185 - val_accuracy: 0.9710
Epoch 4/50
213750/213750 [==============================] - 962s 5ms/step - loss: 0.1030 - accuracy: 0.9775 - val_loss: 0.0809 - val_accuracy: 0.9862
Epoch 5/50
213750/213750 [==============================] - 959s 4ms/step - loss: 0.0783 - accuracy: 0.9866 - val_loss: 0.0775 - val_accuracy: 0.9860
Epoch 6/50
213750/213750 [==============================] - 956s 4ms/step - loss: 0.0609 - accuracy: 0.9924 - val_loss: 0.0504 - val_accuracy: 0.9958
Epoch 7/50
213750/213750 [==============================] - 954s 4ms/step - loss: 0.0493 - accuracy:

In [ ]:
predicted_labels = model.predict(np.array(x_test))

res = np.array(predicted_labels > 0.5, dtype = int)

res = res.reshape((len(x_test)))
y_test = y_test.reshape((len(y_test)))

total = 0

for i in range(len(res)):
    if res[i] == y_test[i]:
        total += 1

print(total/len(x_test))

2813/2813 [==============================] - 4s 1ms/step
0.9992222222222222
